<a href="https://colab.research.google.com/github/JD32919/EDA/blob/main/Hash_tables_1M_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import string
import time
import sys

# Implementación de MurmurHash3 para cadenas de texto
def murmurhash3_32(key, seed=0):
    key = key.encode('utf-8')
    c1 = 0xcc9e2d51
    c2 = 0x1b873593
    r1 = 15
    r2 = 13
    m = 5
    n = 0xe6546b64
    hash_value = seed

    # Procesar bloques de 4 bytes
    for i in range(0, len(key) // 4 * 4, 4):
        k = key[i] | (key[i + 1] << 8) | (key[i + 2] << 16) | (key[i + 3] << 24)
        k = (k * c1) & 0xFFFFFFFF
        k = (k << r1 | k >> (32 - r1)) & 0xFFFFFFFF
        k = (k * c2) & 0xFFFFFFFF

        hash_value ^= k
        hash_value = (hash_value << r2 | hash_value >> (32 - r2)) & 0xFFFFFFFF
        hash_value = (hash_value * m + n) & 0xFFFFFFFF

    # Procesar los últimos bytes del mensaje
    tail = len(key) & 3
    if tail == 3:
        k1 = (key[len(key) - 3] << 16) | (key[len(key) - 2] << 8) | key[len(key) - 1]
    elif tail == 2:
        k1 = (key[len(key) - 2] << 8) | key[len(key) - 1]
    elif tail == 1:
        k1 = key[len(key) - 1]
    else:
        k1 = 0

    if tail != 0:
        k1 = (k1 * c1) & 0xFFFFFFFF
        k1 = (k1 << r1 | k1 >> (32 - r1)) & 0xFFFFFFFF
        k1 = (k1 * c2) & 0xFFFFFFFF
        hash_value ^= k1

    # Finalización
    hash_value ^= len(key)
    hash_value ^= hash_value >> 16
    hash_value = (hash_value * 0x85ebca6b) & 0xFFFFFFFF
    hash_value ^= hash_value >> 13
    hash_value = (hash_value * 0xc2b2ae35) & 0xFFFFFFFF
    hash_value ^= hash_value >> 16
    return hash_value

# Implementación de la clase HashTable con encadenamiento
class HashTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.table = [[] for _ in range(size)]
        self.collisions = 0
        self.insertions = 0

    def hash_function(self, key):
        return murmurhash3_32(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        if self.table[index]:  # Si ya existe un elemento en el índice, es una colisión
            self.collisions += 1
        self.table[index].append((key, value))
        self.insertions += 1

    def calculate_metrics(self):
        # Factor de carga: Número de claves únicas insertadas / tamaño de la tabla
        load_factor = self.insertions / self.size
        # Porcentaje de colisiones
        collision_percentage = (self.collisions / self.insertions) * 100 if self.insertions > 0 else 0
        # Uso de memoria (en bytes)
        memory_usage = sys.getsizeof(self.table)
        return load_factor, collision_percentage, memory_usage

    def display(self):
        # Mostrar el contenido de la tabla hash
        for i, bucket in enumerate(self.table):
            if bucket:  # Mostrar solo posiciones no vacías
                print(f"Índice {i}: {bucket}")

# Generador de claves aleatorias
def random_key(length=10):
    return ''.join(random.choice(string.ascii_letters) for _ in range(length))

# Ejemplo de uso de la tabla hash
if __name__ == "__main__":
    # Crear una tabla hash con tamaño de exactamente 1 millón
    hash_table = HashTable(size=1_000_000)

    # Medir el tiempo de inicio
    start_time = time.time()

    # Insertar un millón de datos en la tabla
    for _ in range(1_000_000):
        key = random_key()  # Clave aleatoria
        value = random.randint(1, 1_000_000)  # Valor aleatorio
        hash_table.insert(key, value)

    # Medir el tiempo de fin
    end_time = time.time()

    # Calcular métricas
    load_factor, collision_percentage, memory_usage = hash_table.calculate_metrics()
    elapsed_time = end_time - start_time  # Complejidad temporal

    # Mostrar el contenido de la tabla hash
    hash_table.display()

    # Mostrar las métricas
    print("Factor de carga:", load_factor)
    print("Porcentaje de colisiones:", collision_percentage, "%")
    print("Tiempo de ejecución:", elapsed_time, "segundos")
    print("Uso de memoria:", memory_usage, "bytes")
